In [2]:
from sumo_python.helpers.load_experiment_metadata import load_experiment_config

input_path = "/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/"

config, sim_setup = load_experiment_config(config=input_path + 'config.json',
                        sim_setup=input_path + 'simulation_setups.json',)

In [3]:
import numpy as np
import pandas as pd

from sumo_python.helpers.generate_od_xml import generate_od_xml
from sumo_python.simulator.run_sumo_and_parse_output import run_sumo_and_parse_output
from sumo_python.simulator.ground_truth import run_ground_truth_and_parse
from sumo_python.helpers.nrmse_counts import compute_nrmse_counts

df_gt_data = run_ground_truth_and_parse(config, sim_setup)


od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.gt_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml 
Success.
sumo --output-prefix gt_ --ignore-route-errors=true --net-file=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml --routes=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml -b 54000 -e 57600 --additional-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes /Users/rodrse/Documents/DemandCalibration/notebooks

In [4]:
def objective(x: np.array) -> pd.DataFrame:
    """Compute objective variable. This is the NRMSE loss between ground-truth output
    and the output of the simulation output evaluated at x.

    Args:
        x (np.array): Numpy array of counts to generate TAZ file from.

    Returns:
        pd.Dataframe: Data frame of sampling locations and loss esimate.
    """
    prefix_run = 'iter'    
    generate_od_xml(x, config, sim_setup, prefix_run)
    df_simulated = run_sumo_and_parse_output(config, sim_setup, prefix_run)
    loss_estimate = compute_nrmse_counts(df_gt_data, df_simulated)
    df_output = pd.DataFrame(x.reshape(-1, len(x)),columns = ['x1','x2','x3','x4'])
    df_output['loss_estimate'] = loss_estimate
    
    return df_output

## High-Dimensional sample-efficient Bayesian Optimization 

Source: https://botorch.org/tutorials/saasbo

In [5]:
import os
import torch
import pandas as pd

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
dtype = torch.double

### Declare search space
# dimensionality of input space
d = 4 
bounds = torch.tensor([[400,400,400,400],[1000,1000,1000,1000]], device=device, dtype=dtype) 
bounds


tensor([[ 400.,  400.,  400.,  400.],
        [1000., 1000., 1000., 1000.]], dtype=torch.float64)

In [11]:
### Generate batch of policies using Sobol sequences
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize

n0 = 20
dim = 4
sobol = SobolEngine(dim, scramble=True)
x_0 = sobol.draw(n0).to(dtype=dtype)

# map the normalized into the original parameter space
train_X0 = unnormalize(x_0, bounds)

df_0 = [
    objective(x) for x in train_X0.cpu().detach().numpy()
]

df_0 = pd.concat(df_0)
df_0['iter'] = 0


printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_od_trips.trips.xml 
Success.
Loading net-file from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml' ... done (5ms).
Loading additional-files from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml' ... done (1ms).
Loading done.
Simulation version 1.16.0 started with time: 54000.00.


Step #57600.00 (7ms ~= 142.86*RT, ~103428.57UPS, vehicles TOT 2573 ACT 724 BUF 0)         
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 17.24s
 Real time factor: 208.768
 UPS: 101447.575968
Vehicles: 
 Inserted: 2573
 Running: 724
 Waiting: 0
 Teleports: 3 (Jam: 2, Wrong Lane: 1)
Statistics (avg of 1849):
 RouteLength: 4439.77
 Speed: 11.03
 Duration: 510.47
 WaitingTime: 109.69
 TimeLoss: 188.22
 DepartDelay: 0.49

DijkstraRouter answered 2573 queries and explored 10.46 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibr

Step #57600.00 (10ms ~= 100.00*RT, ~85500.00UPS, vehicles TOT 2724 ACT 855 BUF 63)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 14.90s
 Real time factor: 241.595
 UPS: 124036.574727
Vehicles: 
 Inserted: 2724 (Loaded: 2787)
 Running: 855
 Waiting: 63
 Teleports: 9 (Jam: 4, Wrong Lane: 5)
Statistics (avg of 1869):
 RouteLength: 4350.39
 Speed: 12.39
 Duration: 437.30
 WaitingTime: 80.64
 TimeLoss: 122.34
 DepartDelay: 0.49

DijkstraRouter answered 2787 queries and explored 10.53 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documen

Step #57600.00 (8ms ~= 125.00*RT, ~93625.00UPS, vehicles TOT 2774 ACT 749 BUF 457)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 21.92s
 Real time factor: 164.219
 UPS: 92818.949001
Vehicles: 
 Inserted: 2774 (Loaded: 3231)
 Running: 749
 Waiting: 457
 Teleports: 9 (Jam: 7, Wrong Lane: 2)
Statistics (avg of 2025):
 RouteLength: 4273.69
 Speed: 11.95
 Duration: 464.97
 WaitingTime: 105.90
 TimeLoss: 155.38
 DepartDelay: 5.39

DijkstraRouter answered 3464 queries and explored 10.02 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Docume

Step #57600.00 (9ms ~= 111.11*RT, ~99333.33UPS, vehicles TOT 2722 ACT 894 BUF 45)         
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 21.08s
 Real time factor: 170.81
 UPS: 91582.748150
Vehicles: 
 Inserted: 2722 (Loaded: 2767)
 Running: 894
 Waiting: 45
 Teleports: 32 (Jam: 20, Wrong Lane: 12)
Statistics (avg of 1828):
 RouteLength: 4449.93
 Speed: 12.36
 Duration: 439.06
 WaitingTime: 76.74
 TimeLoss: 115.39
 DepartDelay: 0.48

DijkstraRouter answered 2767 queries and explored 10.88 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Docume

...

Step #57600.00 (7ms ~= 142.86*RT, ~119142.86UPS, vehicles TOT 2743 ACT 834 BUF 162)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 15.71s
 Real time factor: 229.124
 UPS: 126092.795316
Vehicles: 
 Inserted: 2743 (Loaded: 2905)
 Running: 834
 Waiting: 162
 Teleports: 31 (Jam: 16, Wrong Lane: 15)
Statistics (avg of 1909):
 RouteLength: 4358.29
 Speed: 11.75
 Duration: 470.57
 WaitingTime: 100.74
 TimeLoss: 154.41
 DepartDelay: 0.49

DijkstraRouter answered 3026 queries and explored 10.26 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Do

Step #57600.00 (9ms ~= 111.11*RT, ~74888.89UPS, vehicles TOT 2597 ACT 674 BUF 0)          
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 18.17s
 Real time factor: 198.118
 UPS: 88631.665841
Vehicles: 
 Inserted: 2597
 Running: 674
 Waiting: 0
 Teleports: 2 (Wrong Lane: 2)
Statistics (avg of 1923):
 RouteLength: 4302.64
 Speed: 11.03
 Duration: 523.29
 WaitingTime: 126.04
 TimeLoss: 210.90
 DepartDelay: 0.49

DijkstraRouter answered 2597 queries and explored 9.70 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/note

Step #57600.00 (10ms ~= 100.00*RT, ~78700.00UPS, vehicles TOT 3139 ACT 787 BUF 196)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 26.23s
 Real time factor: 137.258
 UPS: 90548.650297
Vehicles: 
 Inserted: 3139 (Loaded: 3335)
 Running: 787
 Waiting: 196
 Teleports: 3 (Jam: 1, Wrong Lane: 2)
Statistics (avg of 2352):
 RouteLength: 4401.39
 Speed: 10.41
 Duration: 562.17
 WaitingTime: 95.70
 TimeLoss: 242.50
 DepartDelay: 0.49

DijkstraRouter answered 3383 queries and explored 10.17 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documen

Step #57600.00 (8ms ~= 125.00*RT, ~94625.00UPS, vehicles TOT 2941 ACT 757 BUF 102)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 25.50s
 Real time factor: 141.154
 UPS: 84991.609159
Vehicles: 
 Inserted: 2941 (Loaded: 3043)
 Running: 757
 Waiting: 102
 Teleports: 1 (Wrong Lane: 1)
Statistics (avg of 2184):
 RouteLength: 4331.35
 Speed: 10.54
 Duration: 575.02
 WaitingTime: 129.43
 TimeLoss: 261.01
 DepartDelay: 0.64

DijkstraRouter answered 3165 queries and explored 9.82 edges on average.
DijkstraRouter spent 0.02s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/Deman

Step #57600.00 (10ms ~= 100.00*RT, ~89200.00UPS, vehicles TOT 2974 ACT 892 BUF 253)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 24.88s
 Real time factor: 144.712
 UPS: 89623.869438
Vehicles: 
 Inserted: 2974 (Loaded: 3227)
 Running: 892
 Waiting: 253
 Teleports: 23 (Jam: 11, Wrong Lane: 12)
Statistics (avg of 2082):
 RouteLength: 4410.32
 Speed: 12.04
 Duration: 437.07
 WaitingTime: 73.72
 TimeLoss: 117.68
 DepartDelay: 0.49

DijkstraRouter answered 3228 queries and explored 10.54 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).


In [14]:
from botorch.utils.transforms import normalize

train_X = torch.from_numpy(df_0[["x1","x2","x3","x4"]].values)
train_X_norm = normalize(train_X,bounds)

train_Y = -torch.from_numpy(df_0[["loss_estimate"]].values)


In [16]:
import os

from botorch import fit_fully_bayesian_model_nuts
from botorch.acquisition import qExpectedImprovement
from botorch.models.fully_bayesian import SaasFullyBayesianSingleTaskGP
from botorch.models.transforms import Standardize
from botorch.optim import optimize_acqf

In [17]:
tkwargs = {
    "device": torch.device("cuda:1" if torch.cuda.is_available() else "cpu"),
    "dtype": torch.double,
}

In [18]:
SMOKE_TEST = True

In [19]:
WARMUP_STEPS = 256 if not SMOKE_TEST else 32
NUM_SAMPLES = 128 if not SMOKE_TEST else 16
THINNING = 16

In [21]:
gp = SaasFullyBayesianSingleTaskGP(
    train_X=train_X, 
    train_Y=train_Y, 
    outcome_transform=Standardize(m=1)
)
fit_fully_bayesian_model_nuts(
    gp,
    warmup_steps=WARMUP_STEPS,
    num_samples=NUM_SAMPLES,
    thinning=THINNING,
    disable_progbar=True,
)
with torch.no_grad():
    posterior = gp.posterior(train_X)

In [22]:
print(gp.median_lengthscale.detach())

tensor([ 0.9981,  1.5157, 12.9865,  4.1720], dtype=torch.float64)


Make predictions with the model¶
In the next cell we show how to make predictions with the SAAS model. You compute the mean and variance for test points just like for any other BoTorch posteriors. Note that the mean and posterior will have an extra batch dimension at -3 that corresponds to the number of MCMC samples (which is 16 in this tutorial).

In [23]:
print(posterior.mean.shape)
print(posterior.variance.shape)

torch.Size([1, 20, 1])
torch.Size([1, 20, 1])


## BayesOpt loop

In [27]:
from botorch.optim import optimize_acqf

BATCH_SIZE = 1 
NUM_RESTARTS = 5 
RAW_SAMPLES = 32 
    
def optimize_acqf_and_get_observation(acq_func,bounds):
    """Optimizes the acquisition function, and returns a new candidate."""
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=torch.tensor([[0.0] * dim, [1.0] * dim], device=device, dtype=dtype),
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )

    # observe new values 
    new_x = candidates.detach()
    
    return unnormalize(new_x, bounds)

In [28]:
### Run loop
NITER = 2
output = [df_0]
best_value = []

for i in range(NITER):
    
    print(f"####### {i+1} ########")
    
    df_ = df_0 if i == 0 else df_i
    
    n_obs = df_.shape[0]
    print(f"##### n={n_obs} #####")
    
    # Training data
    train_X = torch.from_numpy(df_[["x1","x2","x3","x4"]].values)    
    train_X_norm = normalize(train_X,bounds)
    
    train_Y = -torch.from_numpy(df_[["loss_estimate"]].values) # Take negative
    
    # best value so far
    best_y = train_Y.max()
    best_value.append(best_y)
    print(f"##### best_value={best_y} #####")

    # Declare model with newest data
    gp = SaasFullyBayesianSingleTaskGP(
        train_X=train_X_norm,
        train_Y=train_Y,
        outcome_transform=Standardize(m=1),
    )

    # Fit model
    fit_fully_bayesian_model_nuts(
        gp,
        warmup_steps=WARMUP_STEPS,
        num_samples=NUM_SAMPLES,
        thinning=THINNING,
        disable_progbar=True,
    )

    # Construct acquistion function 
    EI = qExpectedImprovement(model=gp, best_f=best_y)
    
    # Maximize acquisition function to get next observation
    new_x = optimize_acqf_and_get_observation(acq_func=EI,bounds=bounds)
    
    # Query SUMO on new sampling location
    df_simulation = objective(new_x.cpu().detach().numpy().reshape(-1))
    df_simulation['iter'] = i+1
    
    # Update history of training data
    output.append(df_simulation)
    
    df_i = pd.concat(output)

df = pd.concat(output)


####### 1 ########
##### n=20 #####
##### best_value=-0.05704021241819285 #####
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_od_trips.trips.xml 
Success.
Loading net-file from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml' ... done (9ms).
Loading additional-files from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml' ... done (1ms).
Loadin

Step #57600.00 (9ms ~= 111.11*RT, ~101111.11UPS, vehicles TOT 3066 ACT 910 BUF 334)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 22.25s
 Real time factor: 161.812
 UPS: 105888.079827
Vehicles: 
 Inserted: 3066 (Loaded: 3400)
 Running: 910
 Waiting: 334
 Teleports: 27 (Jam: 14, Wrong Lane: 13)
Statistics (avg of 2156):
 RouteLength: 4453.08
 Speed: 12.50
 Duration: 416.98
 WaitingTime: 63.25
 TimeLoss: 93.41
 DepartDelay: 0.41

DijkstraRouter answered 3401 queries and explored 10.99 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
####### 2 ########
##### n=21 #####
##### best_value=-0.05704021241819285 #####
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo

Step #57600.00 (9ms ~= 111.11*RT, ~82555.56UPS, vehicles TOT 3013 ACT 743 BUF 322)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 23.35s
 Real time factor: 154.156
 UPS: 92089.667280
Vehicles: 
 Inserted: 3013 (Loaded: 3335)
 Running: 743
 Waiting: 322
 Teleports: 7 (Jam: 6, Wrong Lane: 1)
Statistics (avg of 2270):
 RouteLength: 4273.00
 Speed: 10.83
 Duration: 552.91
 WaitingTime: 141.75
 TimeLoss: 243.43
 DepartDelay: 1.58

DijkstraRouter answered 3519 queries and explored 9.56 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).


In [ ]:
X = SobolEngine(dimension=DIM, scramble=True, seed=0).draw(N_INIT).to(**tkwargs)
Y = branin_emb(X).unsqueeze(-1)
print(f"Best initial point: {Y.min().item():.3f}")


for i in range(N_ITERATIONS):
    train_Y = -1 * Y  # Flip the sign since we want to minimize f(x)
    gp = SaasFullyBayesianSingleTaskGP(
        train_X=X,
        train_Y=train_Y,
        train_Yvar=torch.full_like(train_Y, 1e-6),
        outcome_transform=Standardize(m=1),
    )
    fit_fully_bayesian_model_nuts(
        gp,
        warmup_steps=WARMUP_STEPS,
        num_samples=NUM_SAMPLES,
        thinning=THINNING,
        disable_progbar=True,
    )

    
    candidates, acq_values = optimize_acqf(
        EI,
        bounds=torch.cat((torch.zeros(1, DIM), torch.ones(1, DIM))).to(**tkwargs),
        q=BATCH_SIZE,
        num_restarts=10,
        raw_samples=1024,
    )

    Y_next = torch.cat([branin_emb(x).unsqueeze(-1) for x in candidates]).unsqueeze(-1)
    if Y_next.min() < Y.min():
        ind_best = Y_next.argmin()
        x0, x1 = candidates[ind_best, :2].tolist()
        print(
            f"{i + 1}) New best: {Y_next[ind_best].item():.3f} @ "
            f"[{x0:.3f}, {x1:.3f}]"
        )
    X = torch.cat((X, candidates))
    Y = torch.cat((Y, Y_next))